In [1]:
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sn
from tabulate import tabulate
import datetime
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from scipy.optimize import minimize
import matplotlib.patches as mpatches
import plotly.graph_objects as go
from scipy import stats

In [2]:
### Loading the data 
df=pd.read_csv("C:\\Users\\kabra\\OneDrive - SP JAIN SCHOOL OF GLOBAL MANAGEMENT\\Desktop\\umass\\MasterFile9July.csv")
df=df.rename(columns = {'Unnamed: 0':'stock_name'})
df=df.drop('Unnamed: 1',axis=1)
df.Date=pd.to_datetime(df.Date)

,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY
count,5412.00000,5412.000000,5412.000000,5412.000000,5412.000000,5412.000000,5412.000000,5412.000000,5412.000000
mean,30.29733,42.460879,13.631922,34.771489,31.495243,27.867904,27.237603,40.257302,45.966089
std,14.07843,18.234632,5.978272,18.824871,19.914167,14.779145,14.078599,24.255819,30.927839
min,10.85800,12.964000,3.097500,12.065000,8.981000,11.544000,8.009000,16.213000,13.543000
25%,17.59300,21.307500,9.633000,20.160000,17.372250,15.873500,15.703500,22.127500,23.190000
50%,28.42500,44.710000,12.124000,26.990000,22.915000,20.130000,23.270000,26.615000,29.905000
75%,42.00000,57.700000,15.135250,48.262500,39.382500,41.910000,36.452500,62.442500,67.395000
max,60.80000,80.560000,30.710000,84.180000,108.110000,63.930000,69.680000,103.790000,132.970000


### Creating a dataframe with the daily closing prices for all the ETFs

In [6]:
dfclose = df[['Date', 'stock_name', 'Close']]
closing = dfclose.set_index('Date')
closing = closing.pivot(columns='stock_name')
closing.columns = [col[1] for col in closing.columns]
closing
closing.describe()

,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY
count,5412.00000,5412.000000,5412.000000,5412.000000,5412.000000,5412.000000,5412.000000,5412.000000,5412.000000
mean,30.29733,42.460879,13.631922,34.771489,31.495243,27.867904,27.237603,40.257302,45.966089
std,14.07843,18.234632,5.978272,18.824871,19.914167,14.779145,14.078599,24.255819,30.927839
min,10.85800,12.964000,3.097500,12.065000,8.981000,11.544000,8.009000,16.213000,13.543000
25%,17.59300,21.307500,9.633000,20.160000,17.372250,15.873500,15.703500,22.127500,23.190000
50%,28.42500,44.710000,12.124000,26.990000,22.915000,20.130000,23.270000,26.615000,29.905000
75%,42.00000,57.700000,15.135250,48.262500,39.382500,41.910000,36.452500,62.442500,67.395000
max,60.80000,80.560000,30.710000,84.180000,108.110000,63.930000,69.680000,103.790000,132.970000


### Calculating the daily Closing Log returns

In [3]:
c2cl=closing.pct_change().apply(lambda x: np.log(1+x))
c2cl=c2cl.dropna()

### Capping Log Returns to +/- 3 Standard Deviations


In [4]:
etflist=['XLB', 'XLE', 'XLF','XLI', 'XLK', 'XLP', 'XLU', 'XLV', 'XLY']
for etf in etflist:
    st=c2cl[etf].std()
    m=c2cl[etf].mean()
    c2cl[etf][c2cl[etf]<(m-(3*st))]=m-3*st
    c2cl[etf][c2cl[etf]>(m+(3*st))]=m+3*st
c2cl
#c2cl.to_csv("C:\\Users\\kabra\\OneDrive - SP JAIN SCHOOL OF GLOBAL MANAGEMENT\\Desktop\\CLOSE TO CLOSE RETURNS.csv")

,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY
Date,,,,,,,,,
1998-12-23,0.010483,0.020663,0.015256,0.017311,0.024174,0.023328,-0.003674,0.022240,0.003658
1998-12-24,0.022758,-0.005332,0.006615,0.012404,-0.003923,-0.000601,0.018238,0.006091,0.018640
1998-12-28,-0.008797,-0.005291,-0.012544,0.006512,0.002748,-0.006329,-0.005187,-0.014681,-0.008900
1998-12-29,0.018191,0.009941,0.009862,0.013503,0.002349,0.021770,0.016953,0.021942,0.021758
1998-12-30,-0.002921,-0.015260,-0.003933,-0.004481,-0.002741,-0.006231,-0.008158,-0.008476,-0.008389
...,...,...,...,...,...,...,...,...,...
2020-07-02,0.018600,0.011583,0.001525,0.008589,0.005429,0.004928,0.002939,0.007410,0.004022
2020-07-06,0.014498,0.002942,0.019184,0.014248,0.017234,0.004735,-0.012859,0.010185,0.023197
2020-07-07,0.000000,-0.032292,-0.020708,-0.019334,-0.010041,0.009404,-0.003855,-0.008808,-0.016350


### Calculating the daily closing returns for all the ETFs

In [5]:
c2c=closing.pct_change()
c2c=c2c.dropna()
c2c.describe()

,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY
count,5411.000000,5411.000000,5411.000000,5411.000000,5411.000000,5411.000000,5411.000000,5411.000000,5411.000000
mean,0.000398,0.000326,0.000349,0.000461,0.000407,0.000289,0.000338,0.000385,0.000599
std,0.015601,0.018107,0.019488,0.020700,0.016514,0.009987,0.012531,0.011848,0.024622
min,-0.115030,-0.201316,-0.164703,-0.487058,-0.138241,-0.093645,-0.113680,-0.099378,-0.582112
25%,-0.007049,-0.008211,-0.007110,-0.005725,-0.006382,-0.004327,-0.005481,-0.005160,-0.005776
50%,0.000657,0.000511,0.000479,0.000704,0.000931,0.000455,0.000946,0.000692,0.000783
75%,0.008193,0.009561,0.007862,0.007022,0.007579,0.005141,0.006610,0.006313,0.007249
max,0.121800,0.202932,0.307198,0.962298,0.171466,0.093349,0.126270,0.131194,1.359243
